In [ ]:
#!pip install pycaret

In [1]:
import os
import pandas as pd
from pycaret.classification import *

In [2]:
directory= 'C:\\Users\\USER\\PycharmProjects\\ERC\\'
os.chdir(directory)

In [3]:
data_n = 'temp' #temp, ibi, eda
method = 'tsne' #pca, tsne, lle, umap

In [4]:
data = pd.read_csv('data/{}_{}.csv'.format(data_n, method))                  

In [5]:
clf = setup(data, target='Emotion')
best_model = compare_models(sort='F1', n_select=3)

,Description,Value
0,Session id,1945
1,Target,Emotion
2,Target type,Multiclass
3,Original data shape,"(296, 4)"
4,Transformed data shape,"(296, 4)"
5,Transformed train set shape,"(207, 4)"
6,Transformed test set shape,"(89, 4)"
7,Numeric features,3
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.2471,0.5118,0.2471,0.2306,0.2303,0.0540,0.0553,0.2490
gbc,Gradient Boosting Classifier,0.2276,0.5068,0.2276,0.2457,0.2241,0.0319,0.0332,0.2030
rf,Random Forest Classifier,0.2271,0.5027,0.2271,0.2359,0.2161,0.0289,0.0305,0.1330
xgboost,Extreme Gradient Boosting,0.2324,0.5053,0.2324,0.2264,0.2133,0.0355,0.0382,0.1110
nb,Naive Bayes,0.2214,0.4899,0.2214,0.2067,0.2034,0.0126,0.0133,0.0730
qda,Quadratic Discriminant Analysis,0.2362,0.4882,0.2362,0.2102,0.1998,0.0281,0.0330,0.0740
et,Extra Trees Classifier,0.2176,0.5130,0.2176,0.2016,0.1991,0.0170,0.0180,0.1330
lr,Logistic Regression,0.2319,0.4828,0.2319,0.1946,0.1987,0.0220,0.0230,0.6590
dt,Decision Tree Classifier,0.2029,0.5007,0.2029,0.2227,0.1976,0.0026,0.0021,0.0720
lda,Linear Discriminant Analysis,0.2319,0.4818,0.2319,0.1905,0.1969,0.0222,0.0234,0.0780


Processing:   0%|          | 0/71 [00:00<?, ?it/s]

In [14]:
new_data = data.drop('Emotion', axis=1)
prediction = predict_model(best_model[0], data=new_data)
prediction

,Sex,0,1,prediction_label,prediction_score
0,0,-9.183681,-12.407987,1,0.6325
1,1,-7.848112,-10.861624,0,0.5067
2,1,5.016291,4.994897,2,0.3142
3,1,9.581699,7.548366,0,0.6391
4,1,1.952928,-2.064449,0,0.5598
...,...,...,...,...,...
291,0,15.606421,11.039059,1,0.5186
292,1,16.140240,10.545608,1,0.6995
293,0,-3.395632,-3.548883,2,0.2783
294,1,-0.484301,-3.478823,1,0.4673
